In [ ]:
#Primary Modules
import pandas as pd
import numpy as np

#Model_selection Tools
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score 

#Model
from xgboost import XGBClassifier as xgb
from xgboost import XGBRFClassifier as xgb_rf

#Grid Search for Optimal Hyperparameters
from sklearn.model_selection import RandomizedSearchCV

#Data Object Output
import pickle


In [ ]:
# Import Modified Data

train_data=pd.read_pickle(r"/content/drive/My Drive/Colab Data/Pet Adoption/modified_train.pickle")
test_data=pd.read_pickle(r"/content/drive/My Drive/Colab Data/Pet Adoption/modified_test.pickle")

In [ ]:
Final_features_b = ['condition','color_type','length(m)','height(cm)','X1','X2','diff']
Final_features_p = ['condition_impute','color_type','pet_pred','X1','X2','diff']


X_b= train_data[Final_features_b].copy()
X_p= train_data[Final_features_p].copy()


In [ ]:
y_b=train_data['breed_category'].to_numpy()
y_b = y_b.reshape(y_b.shape[0], )
y_b=pd.Series(y_b)

y_p=train_data['pet_category'].to_numpy()
yd_p = y_p.reshape(y_p.shape[0], )
y_p=pd.Series(y_p)

In [ ]:
X_b_train,X_b_valid,y_b_train,y_b_valid=train_test_split(X_b,y_b,test_size=0.3,random_state=21)
X_p_train,X_p_valid,y_p_train,y_p_valid=train_test_split(X_p,y_p,test_size=0.3,random_state=21)


X_b_test=test_data[Final_features_b].copy()
X_p_test=test_data[Final_features_p].copy()

In [ ]:
#XGB Benchmark



benchmark_model=xgb(n_estimators=1000,random_state=21,gamma=30,objective='multi:softmax',num_class=3,
                    subsample=0.8,sampling_method='gradient_based',sample_type='weighted',updater='grow_gpu_hist,prune',
                    predictor='gpu_predictor',eval_metric=['merror','mlogloss'])

benchmark_model.fit(X_b_train, y_b_train, verbose=False)
pred=benchmark_model.predict(X_b_valid)
        
#y_b_valid=y_b_valid.to_numpy()
f1=f1_score(y_b_valid,pred,average="weighted")

print(f1)


benchmark_model=xgb(n_estimators=1000,random_state=21,gamma=10,
                    subsample=0.8,sampling_method='gradient_based'
                    ,objective='multi:softmax',num_class=4,sample_type='weighted',max_bin=52,updater='grow_gpu_hist,refresh',
                    predictor='gpu_predictor',eval_metric=['merror','mlogloss']
                    )


benchmark_model.fit(X_p_train, y_p_train,verbose=False)
pred=benchmark_model.predict(X_p_valid)

#y_p_valid=y_p_valid.to_numpy()
f1=f1_score(y_p_valid,pred,average="weighted")

print(f1)

#0.8978817692842459
#0.898401342222935



[11:15:29] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
0.8993325962212914
[11:15:41] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
0.8984735205038152


In [ ]:
#XGB Benchmark

benchmark_model=xgb_rf(n_estimators=1000,random_state=21,gamma=30,objective='multi:softmax',num_class=3,
                    subsample=0.8,sampling_method='gradient_based',sample_type='weighted',updater='grow_gpu_hist,prune',
                    predictor='gpu_predictor',eval_metric=['merror','mlogloss'])


benchmark_model.fit(X_b_train, y_b_train, verbose=False)
pred=benchmark_model.predict(X_b_valid)
        
#y_b_valid=y_b_valid.to_numpy()
f1=f1_score(y_b_valid,pred,average="weighted")

print(f1)


benchmark_model=xgb_rf(n_estimators=1000,random_state=21,gamma=10,
                    subsample=0.8,sampling_method='gradient_based'
                    ,objective='multi:softmax',num_class=4,sample_type='weighted',max_bin=52,updater='grow_gpu_hist,refresh',
                    predictor='gpu_predictor',eval_metric=['merror','mlogloss']
                    )


benchmark_model.fit(X_p_train, y_p_train,verbose=False)
pred=benchmark_model.predict(X_p_valid)

#y_p_valid=y_p_valid.to_numpy()
f1=f1_score(y_p_valid,pred,average="weighted")

print(f1)

[11:16:07] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
0.8882421664401402
[11:16:18] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
0.8535974540016406


In [ ]:
# Create the parameter grid: xgb_param_grid 

xgb_param_grid = {
    'n_estimators': range(450,600,25),
    'max_depth': range(3,5),
    'reg_lambda': np.linspace(0.3,0.6,3),
    'reg_alpha': np.linspace(0.6,0.9,3),
    'gamma': [30],
    'learning_rate': np.linspace(0.05,0.25,4),
    'max_bin':[64,96],
    'early_stopping_rounds': [50],
    'n_jobs':[4]
}


# Instantiate the regressor: gbm
xgbc_random = xgb(random_state=21,objective='multi:softmax',num_class=3
                    ,sampling_method='gradient_based',sample_type='weighted',updater='grow_gpu_hist,prune',
                    predictor='gpu_predictor',eval_metric=['merror','mlogloss'])

#Perform random search: grid_mse
xgb_random = RandomizedSearchCV(param_distributions=xgb_param_grid, estimator = xgbc_random,verbose=20, scoring = "f1_weighted", n_iter = 5,cv=2,n_jobs=4)


#Fit randomized_mse to the data
xgb_random.fit(X_b, y_b)

#Print the best parameters and lowest RMSE
print("Best parameters found: ", xgb_random.best_params_)
print("Best f1_weighted found: ", xgb_random.best_score_)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:   23.5s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:   23.9s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:   24.9s
[Parallel(n_jobs=4)]: Done   4 out of  10 | elapsed:   25.0s remaining:   37.5s
[Parallel(n_jobs=4)]: Done   5 out of  10 | elapsed:   48.1s remaining:   48.1s
[Parallel(n_jobs=4)]: Done   6 out of  10 | elapsed:   48.7s remaining:   32.5s
[Parallel(n_jobs=4)]: Done   7 out of  10 | elapsed:   52.6s remaining:   22.5s
[Parallel(n_jobs=4)]: Done   8 out of  10 | elapsed:   53.3s remaining:   13.3s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  1.0min remaining:    0.0s
[Parallel(n_jobs=4)]: Done  10 out of  10 | elapsed:  1.0min finished


[11:19:24] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
Best parameters found:  {'reg_lambda': 0.6, 'reg_alpha': 0.6, 'n_jobs': 4, 'n_estimators': 525, 'max_depth': 3, 'max_bin': 96, 'learning_rate': 0.05, 'gamma': 30, 'early_stopping_rounds': 50}
Best f1_weighted found:  0.8876560065290834


In [ ]:
#Best parameters found:  {'reg_lambda': 0.3, 'reg_alpha': 0.75, 'n_jobs': 4, 'n_estimators': 475, 'max_depth': 10, 'max_bin': 64, 'learning_rate': 0.25, 'gamma': 10, 'early_stopping_rounds': 50}
#Best f1_weighted found:  0.9035719311984693
model=xgb(reg_lambda =0.3, reg_alpha = 0.75, n_jobs = 4, n_estimators = 475, 
          max_depth = 10, max_bin = 64, learning_rate = 0.25, 
          gamma =10, early_stopping_rounds = 50)

model.fit(X_b, y_b)
predic_breed = model.predict(X_b_test)

In [ ]:
predic_breed

array([1., 0., 0., ..., 1., 2., 1.])

In [ ]:

# Create the parameter grid: gbm_param_grid 

xgb_param_grid = {
    'n_estimators': range(128,1024,32),
    'max_depth': range(6,10),
    'reg_lambda': np.linspace(0.1,1,9),
    'reg_alpha': np.linspace(0.1,1,9),
    'gamma': [10,15,20,25],
    'learning_rate': np.linspace(0.05,0.25,4),
    'subsample': [0.7,0.8,0.9,1],
    'early_stopping_rounds': [50],
    'n_jobs':[4]
}

# Instantiate the regressor: gbm
xgbc_random = xgb(random_state=21,sampling_method='gradient_based'
                  ,objective='multi:softmax',num_class=4,sample_type='weighted',max_bin=52,updater='grow_gpu_hist,refresh',
                  predictor='gpu_predictor',eval_metric=['merror','mlogloss']
                    )
#Perform random search: grid_mse
xgb_random = RandomizedSearchCV(param_distributions=xgb_param_grid,estimator = xgbc_random,verbose=20, scoring = "f1_weighted", n_iter = 50,
                                cv=2,n_jobs=4)


#Fit randomized_mse to the data
xgb_random.fit(X_p, y_p,verbose=False)

#Print the best parameters and lowest RMSE
print("Best parameters found: ", xgb_random.best_params_)
print("Best f1_weighted found: ", xgb_random.best_score_)

Fitting 2 folds for each of 50 candidates, totalling 100 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:  3.7min
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:  3.8min
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:  5.2min
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:  7.4min
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:  7.6min
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed: 10.8min
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed: 10.9min
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed: 11.0min
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed: 11.2min
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed: 12.0min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed: 12.2min
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed: 13.6min
[Parallel(

[10:53:13] WARNING: /workspace/src/learner.cc:194: DANGER AHEAD: You have manually specified `updater` parameter. The `tree_method` parameter will be ignored. Incorrect sequence of updaters will produce undefined behavior. For common uses, we recommend using `tree_method` parameter instead.
Best parameters found:  {'subsample': 0.9, 'reg_lambda': 0.4375, 'reg_alpha': 0.325, 'n_jobs': 4, 'n_estimators': 288, 'max_depth': 7, 'learning_rate': 0.05, 'gamma': 10, 'early_stopping_rounds': 50}
Best f1_weighted found:  0.8983068709667745


In [ ]:
#88.7 
model=xgb(subsample = 0.9,reg_lambda =0.4375, reg_alpha = 0.325, n_jobs = 4, n_estimators = 288, 
          max_depth = 7, max_bin = 64, learning_rate = 0.05, 
          gamma =10, early_stopping_rounds = 50)

model.fit(X_p, y_p)
predic_pet = model.predict(X_p_test)

In [ ]:
predic_pet = predic_pet.reshape(predic_pet.shape[0], )
predic_pet

array([2, 1, 2, ..., 2, 4, 2])

In [ ]:
output = pd.DataFrame({'pet_id': test_data.pet_id, 'breed_category': predic_breed,'pet_category':predic_pet})
output.to_csv('my_submission_71.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
